# Testing/debugging YSSMB simulations

This notebook contains a systematic test of the functions and patterns used to obtain my (so far very wrong) results of the YSSMB model. These functions are mostly contained in `../HoppingMasterEquation.jl`.



In [1]:

include("../HoppingMasterEquation.jl")

using LinearAlgebra, .HoppingMasterEquation, GLMakie


InterruptException: InterruptException:

## 1. Neighbour indices

In [2]:
include("../HoppingMasterEquation.jl")

using LinearAlgebra, Random, .HoppingMasterEquation, GLMakie

N1 = 4;
N2 = 2;
N2 = 2;

a = 10;

pos = zeros(N1,N2,N2,3);
for i=1:N1
    for j=1:N2
        for k=1:N2
            pos[i,j,k,:] = [i,j,k] .* a;
        end
    end
end

pos = reshape(pos,N1*N2*N2,3);
innn = get_nnn_inds_3d(pos,a);

println("MAX NNN distance = $(sqrt(2)*a)")

for i=1:16
    println("*** $i ***")
    println("pos[$i] = $(pos[i,:])")
    println("NNN inds = $(innn[i,:])")
    for j in innn[i,:]
        if j != 0
            println("$j: $(pos[j,:])  $(norm(pos[i,:] - pos[j,:])) ")
        end
    end
    println("\n")
end


## 2.Rate matrix evaluation

In [18]:
include("../HoppingMasterEquation.jl")

using LinearAlgebra, Random, .HoppingMasterEquation, CairoMakie

N1 = 16
N2 = 8
a = 10

println("Defining pos")

pos = zeros(N1,N2,N2,3);
for i=1:N1
    for j=1:N2
        for k=1:N2
            pos[i,j,k,:] = [i,j,k] .*a ;
        end
    end
end

pos = reshape(pos,N1*N2*N2,3);

println("Getting innn")
innn = get_nnn_inds_3d(pos, a)


println("Getting energies")
energies = randn(N1*N2*N2);
energies0 = zeros(N1*N2*N2);

dX = N1 - 1
E = [1,0,0] / dX
T = 300


print("Getting rate matrices")
K =miller_abrahams_YSSMB(pos,energies, innn, T, E)
K0 =miller_abrahams_YSSMB(pos,energies0, innn, T, E)


N = size(pos,1)
plot_data = ones(N*(N-1)÷2,2) .* -1
k = 1

println("Getting plot_data")
for i=1:N
    for j=1:i-1
        ΔR = norm(pos[i,:] - pos[j,:])
        if ΔR ≤ sqrt(2)
            println(k)
            global plot_data[k,1] = ΔR
            global plot_data[k,2] = K0[i,j]
            k+=1
        end
    end
end
println(plot_data)
plot_data = plot_data[plot_data[:,1].> 0,:]
expected = exp.(plot_data[:,1].*-1)

f,ax, sc1 = scatter(plot_data[:,1], plot_data[:,2], color= :black, markersize=7)
lines!(ax,plot_data[:,1], expected, color= :red)

f

In [17]:
plot_data

0×2 Matrix{Float64}